In [1]:
import pandas as pd, os, json

In [2]:
ROOT_FOLDER = "/local/scratch/clmn1/octree_study_new/Experiments"

In [3]:
task = "prostate"
task = "peso"

In [4]:
def get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay):

    if task == "prostate":
        if ema_decay == 0.99:
            path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_OctreeNCASegmentation")
        else:
            path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_{ema_decay}_OctreeNCASegmentation")
    elif task == "peso":
        path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_{ema_decay}_OctreeNCA2DSegmentation")
        

    #if task == "prostate" and ema_decay == 0.99:
    #    path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_OctreeNCASegmentation")
    #else:
    #    path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_{ema_decay}_OctreeNCASegmentation")


    mem_in_gb = json.load(open(os.path.join(path, "mem_allocation.json")))["byte"] / 1000 / 1000 / 1000

    if not os.path.exists(os.path.join(path, "eval", "standard.csv")):
        return {
            "D": None,
            "M": mem_in_gb
        }
    seg_results = pd.read_csv(os.path.join(path, "eval", "standard.csv"), sep="\t")
    mean_dice = seg_results.mean(numeric_only=True)#["DiceScore/0"]

    mean_dice = mean_dice.filter(like="DiceScore").mean()

    return{
        "D": mean_dice,
        "M": mem_in_gb
    }

In [5]:
evaluations = []
steps = 10
alpha = 1.0
model = 16
batch = 3
dice_weight = 1.0
ema_decay = 0.99

for norm in ["none", "batch", "group"]:
    res = get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay)
    res["ablation"] = norm
    evaluations.append(res)
norm = "none"

for steps in [5, 10, 20]:
    res = get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay)
    res["ablation"] = steps
    evaluations.append(res)
steps = 10

for alpha in [1.0, 1.5, 2.0]:
    res = get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay)
    res["ablation"] = alpha
    evaluations.append(res)
alpha = 1.0

for model in [12, 16, 20]:
    res = get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay)
    res["ablation"] = model
    evaluations.append(res)
model = 16

for batch in [1, 3, 5]:
    res = get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay)
    res["ablation"] = batch
    evaluations.append(res)
batch = 3

for dice_weight in [0.0, 1.0, 1.2, 1.5]:
    res = get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay)
    res["ablation"] = dice_weight
    evaluations.append(res)
dice_weight = 1.0

for ema_decay in [0.0, 0.9, 0.99]:
    res = get_results(norm, steps, alpha, model, batch, dice_weight, ema_decay)
    res["ablation"] = ema_decay
    evaluations.append(res)
ema_decay = 0.99

In [ ]:
evaluations = pd.DataFrame(evaluations)
evaluations['D'] = evaluations['D'].apply(lambda x: round(100 * x, 1))
evaluations['M'] = evaluations['M'].apply(lambda x: round(x, 1))
print(evaluations)

       D    M ablation
0   86.3  2.4     none
1   84.5  3.5    batch
2   86.5  3.5    group
3   85.6  1.4        5
4   86.3  2.4       10
5   86.6  4.6       20
6   86.3  2.4      1.0
7   86.2  2.4      1.5
8   86.3  2.4      2.0
9   85.8  1.8       12
10  86.3  2.4       16
11  86.6  3.3       20
12  85.6  0.9        1
13  86.3  2.4        3
14  86.4  3.9        5
15  86.1  2.3      0.0
16  86.3  2.4      1.0
17  86.3  2.3      1.2
18  86.3  2.3      1.5
19  86.1  2.3      0.0
20  86.3  2.3      0.9
21  86.3  2.4     0.99
